In [12]:
#Dataset from https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from os.path import dirname, abspath

In [13]:
print(os.getcwd())

c:\Users\arant\Documents\heart_failure\eda notebooks


In [14]:
base_path = dirname(abspath(os.getcwd()))
data_folder = 'data'
excelfile = 'Datos.xlsx'
pathtoexcel = os.path.join(base_path, data_folder, excelfile)

In [15]:
pathtoexcel

'c:\\Users\\arant\\Documents\\heart_failure\\data\\Datos.xlsx'

In [17]:
data = pd.read_excel (pathtoexcel)

data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,15,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0,Up,0


In [11]:
#This is only a test to split training and test set

from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(data, test_size=0.2, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 734
No. of testing examples: 184


In [18]:
#Fairness on splitting target variable 



In [19]:
#Find variables with missing data
data.isnull().any()

#There is not missing data 

Age               False
Sex               False
ChestPainType     False
RestingBP         False
Cholesterol       False
FastingBS         False
RestingECG        False
MaxHR             False
ExerciseAngina    False
Oldpeak           False
ST_Slope          False
HeartDisease      False
dtype: bool

In [20]:
col = "HeartDisease"
heart = data.loc[:, data.columns != col]
heart.head()
heart

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289,0,Normal,172,N,0,Up
1,49,F,NAP,160,180,0,Normal,156,N,1,Flat
2,37,M,ATA,130,283,0,ST,98,N,0,Up
3,48,F,ASY,138,214,0,Normal,108,Y,15,Flat
4,54,M,NAP,150,195,0,Normal,122,N,0,Up
...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,12,Flat
914,68,M,ASY,144,193,1,Normal,141,N,34,Flat
915,57,M,ASY,130,131,0,Normal,115,Y,12,Flat
916,57,F,ATA,130,236,0,LVH,174,N,0,Flat


In [22]:
heart_encoded = pd.get_dummies(heart, columns = ['Sex', 'ChestPainType','RestingECG',
                                                            'ExerciseAngina', 'ST_Slope'])

heart_encoded


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,15,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,12,0,1,0,0,0,1,0,1,0,1,0,0,1,0
914,68,144,193,1,141,34,0,1,1,0,0,0,0,1,0,1,0,0,1,0
915,57,130,131,0,115,12,0,1,1,0,0,0,0,1,0,0,1,0,1,0
916,57,130,236,0,174,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0


In [23]:
target = data["HeartDisease"]
target.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [24]:
#Number of class I have 
data['HeartDisease'].value_counts()

#Class type 0-> 410  Not heart failure 
#Class type 1-> 508  Heart failure 

1    508
0    410
Name: HeartDisease, dtype: int64

In [25]:
#Separate categorical and numerical variables that needs to be transformed

numeric_heart = heart.select_dtypes(include=[np.number])
categorical_heart = heart.select_dtypes(exclude=[np.number])

numeric_heart.shape[1]
categorical_heart.shape[1]

#5 columns categoricas y #6 columns numerics 

print(numeric_heart)
#print(categorical_heart)

     Age  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak
0     40        140          289          0    172        0
1     49        160          180          0    156        1
2     37        130          283          0     98        0
3     48        138          214          0    108       15
4     54        150          195          0    122        0
..   ...        ...          ...        ...    ...      ...
913   45        110          264          0    132       12
914   68        144          193          1    141       34
915   57        130          131          0    115       12
916   57        130          236          0    174        0
917   38        138          175          0    173        0

[918 rows x 6 columns]


In [26]:
#Transformer requires Index type, so this would be the proper way to do it

categorical_features = heart.select_dtypes(include="object").columns
integer_features = heart.select_dtypes(exclude="object").columns

In [27]:
print(categorical_features)
print(integer_features)


Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')
Index(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak'], dtype='object')


In [28]:
#https://johaupt.github.io/blog/columnTransformer_feature_names.html

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [30]:
#More attributes in the pipeline can be considered
num_pipeline = Pipeline([
               ('std_scaler', StandardScaler()),
    ])
#heart_num_tr = num_pipeline.fit_transform(#place numerical data here)

In [31]:
from sklearn.compose import ColumnTransformer
num_attribs = integer_features
cat_attribs = categorical_features
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

heart_prepared = full_pipeline.fit_transform(heart)

In [32]:
print(heart_prepared)

[[-1.4331398   0.41090889  0.82507026 ...  0.          0.
   1.        ]
 [-0.47848359  1.49175234 -0.17196105 ...  0.          1.
   0.        ]
 [-1.75135854 -0.12951283  0.7701878  ...  0.          0.
   1.        ]
 ...
 [ 0.37009972 -0.12951283 -0.62016778 ...  0.          1.
   0.        ]
 [ 0.37009972 -0.12951283  0.34027522 ...  0.          1.
   0.        ]
 [-1.64528563  0.30282455 -0.21769643 ...  0.          0.
   1.        ]]


In [33]:
names = get_feature_names(full_pipeline)

C:\Users\arant\AppData\Local\Temp\ipykernel_15512\3074341067.py:32: UserWarning: Transformer std_scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
c:\Users\arant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
print(names)

['num__Age', 'num__RestingBP', 'num__Cholesterol', 'num__FastingBS', 'num__MaxHR', 'num__Oldpeak', 'cat__x0_F', 'cat__x0_M', 'cat__x1_ASY', 'cat__x1_ATA', 'cat__x1_NAP', 'cat__x1_TA', 'cat__x2_LVH', 'cat__x2_Normal', 'cat__x2_ST', 'cat__x3_N', 'cat__x3_Y', 'cat__x4_Down', 'cat__x4_Flat', 'cat__x4_Up']


In [35]:
#Create a dataframe with their columns name 

features = pd.DataFrame(heart_prepared,columns=names)
print(features)

     num__Age  num__RestingBP  num__Cholesterol  num__FastingBS  num__MaxHR  \
0   -1.433140        0.410909          0.825070       -0.551341    1.382928   
1   -0.478484        1.491752         -0.171961       -0.551341    0.754157   
2   -1.751359       -0.129513          0.770188       -0.551341   -1.525138   
3   -0.584556        0.302825          0.139040       -0.551341   -1.132156   
4    0.051881        0.951331         -0.034755       -0.551341   -0.581981   
..        ...             ...               ...             ...         ...   
913 -0.902775       -1.210356          0.596393       -0.551341   -0.188999   
914  1.536902        0.627078         -0.053049        1.813758    0.164684   
915  0.370100       -0.129513         -0.620168       -0.551341   -0.857069   
916  0.370100       -0.129513          0.340275       -0.551341    1.461525   
917 -1.645286        0.302825         -0.217696       -0.551341    1.422226   

     num__Oldpeak  cat__x0_F  cat__x0_M  cat__x1_AS

In [36]:
x = features
y = target

In [37]:
#Balance for test and training set 

#Random state
#Controls the shuffling applied to the data before applying the split. 
#Pass an int for reproducible output across multiple function calls.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

#Now dataset is balanced 

#Training target statistics: Counter({1: 403, 0: 331})
#Testing target statistics: Counter({1: 105, 0: 79})

Training target statistics: Counter({1: 403, 0: 331})
Testing target statistics: Counter({1: 105, 0: 79})


In [38]:
print(y)

0      0
1      1
2      0
3      1
4      0
      ..
913    1
914    1
915    1
916    1
917    0
Name: HeartDisease, Length: 918, dtype: int64
